In [1]:
import os
git_dir = os.path.dirname(os.getcwd())
project_dir = os.path.join(git_dir,'zavmo')
os.chdir(project_dir)

In [2]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [3]:
import json
from helpers.chat import get_openai_completion,  get_prompt, make_structured_call
from helpers.functions import parse_type
from IPython.display import Markdown

In [4]:
import openai
from openai import OpenAI
from helpers.functions import create_pydantic_model, get_yaml_data, create_system_message
openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [5]:
stage_name = 'profile'

In [6]:
stage_conf   = get_yaml_data(stage_name)
resp_schema  = stage_conf['response']
resp_model   = create_pydantic_model(name=resp_schema['title'],
                                    description=resp_schema['description'],
                                    fields=resp_schema['fields'])

In [12]:
required_fields = stage_conf['required']

In [13]:
probe_system_message   = create_system_message(stage_name, stage_conf, mode='probe')

In [14]:
user_input = f'Send a personalized welcome message to the learner.'

In [15]:
stage_data = {}

In [16]:
user_content      = f"""The learner is at the {stage_name} stage.
Here's what we know about the user already: {stage_data}
We need to probe the user for the following: {required_fields}

Learner's message: {user_input}
"""
Markdown(user_content)
message_history = []
probe_payload  = create_message_payload(user_input, probe_system_message, message_history, max_tokens=10000)
resp_tool = force_tool_call(resp_model, probe_payload)
resp_tool

The learner is at the profile stage.
Here's what we know about the user already: {}
We need to probe the user for the following: ['first_name', 'last_name', 'age', 'edu_level', 'current_role']

Learner's message: Send a personalized welcome message to the learner.


In [17]:
from helpers.chat import create_message_payload, force_tool_call

Response(action=<ActionEnum.probe: 'probe'>, message="Hi there! 😊 I'm Zavmo, your friendly learning assistant! What's your first name?")